## Import all the relevant Modules 

In [1]:
%matplotlib inline


#TODO: Rename module shortcuts such as hb into something usefull after development
import Modules.Hubbard_Model as hb
from Modules.General.Module_Widgets_and_Sliders import Text_Box, Save_Figure_Button, Click_Save_Figure, set_filename

# default Jupyter widgets
import ipywidgets as widgets
from ipywidgets import HBox, VBox

# Plotting
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark')
plt.rcParams.update({'font.size':16})

# for printlenght and save figures button
import functools
import numpy as np
np.set_printoptions(linewidth=150) #set output length, default=75

def close_widgets(DIR):
    """ Close all widgets `wi` and displays `di` in notebook directory `dir()`.
    """
    for i in range(100):
        if f"w{i}" in DIR:
            exec(f"w{i}.close()")
        if f"d{i}" in DIR:
            exec(f"d{i}.close()")
            

%load_ext autoreload
%autoreload 2

# Lattice Quantum Field Theory (QFT)
<!---  Define a few convenience macros for bra-ket notation. -->
$\newcommand{\ket}[1]{\left\vert{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right\vert}$
$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\dyad}[2]{\left|{#1}\middle\rangle\middle\langle{#2}\right|}$
$\newcommand{\mel}[3]{\left\langle{#1}\vphantom{#2#3}\right|{#2}\left|{#3}\vphantom{#1#2}\right\rangle}$
$\newcommand{\expval}[1]{\left\langle{#1}\right\rangle}$
$\newcommand\dif{\mathop{}\!\mathrm{d}}$
$\newcommand\ii{\mathrm{i}}$
$\newcommand{\coloneqq}{\mathop{:=}}$
$\newcommand{\abs}[1]{\left\vert{#1}\right\vert}$

We will now use methods from QFT to describe the dynamics of a quantum system on lattice. Contrary to the previous notebooks, where we used QM to describe each particles path through space, we will now use QFT to describe the states of space points. This naturally leads to second quantization, a method in which we do not ask "which particles are in which states," but rather "how many particles are in each state".

To simplify our model, we assume our electrons to live on discrete points in space, i.e. on a lattice. Further, as we deal with fermions, the Pauli exclusion principle dictates that a single site can be occupied by no more than two electrons (one with spin up and one with spin down). Consequently, each site can be in one of four states: zero electrons, one up, one down, or up and down. Thus the total number of possible states grows exponentially with the number of sites $n$, as $4^n$. These states are called Fock states or states in the occupation number basis.

# Hubbard model

We use the so called Hubbard model to describe the dynamics of a system of electrons on a lattice. The model's Hamiltonian is defined as follows:

$$ \large
H = t \sum_{\langle ij \rangle, \sigma}  c_{i,\sigma}^\dagger c_{j,\sigma} + U \sum_{i} n_{i \uparrow} n_{i \downarrow},
$$
where $\langle ij \rangle$ denotes nearest neighbor hopping, the spin $\sigma \in \{\frac{1}{2}, -\frac{1}{2}\} = \{\uparrow, \downarrow\}$, $\large c_{i,\sigma}^\dagger$ and $\large c_{j,\sigma}$ is an electron creation and annihilation operator, respectively, and $\large n_{i \uparrow}$ and $\large n_{i \downarrow}$ are the occupation number operators. The latter are zero if no electron with spin $\sigma$ is at site $i$ and one else.

The first term describes electron hopping from site $j$ to $i$ with coupling $t$ (hopping amplitude). The second term models the electron-electron repulsion at site $i$ with on-site interaction strength $U$.

# The Fock Basis states 
Our Hamiltonian $H$ has an equal number of creation and annihilation operators in each term, implying that we cannot change the number of electrons in our system when acting with $H$. Almost the same reasoning ($\sigma$ does not change via $c^\dagger$ and $c$) individually conserves the number of spin up and down electrons. We can therefore reduce the number of Fock states by considering only those with a fixed number of spin up and down electrons (e.g. for diagonalizing $H$). Hence we only need to consider $\large \binom{n}{s_\mathrm{up}} \binom{n}{s_\mathrm{down}}$ basis states instead of $4^n$

Numerically we can label a basis state via a $2n$-dimensional binary array, where the first $n$ indices label the spin up states of site $i$ and the second $n$ label the spin down sites. For example the basis vector $[0,0,1,0,1,1]$ would correspond to a $n=3$ sited problem with one up electron on site 1 and two down electrons on sites 2 and 3.

In the Widget below the total number of Fock basis states as well an individual one are displayed. One can change the number of sites $n \leq 7$, the number of electrons per site $s_\mathrm{up}$ and $s_\mathrm{down}$ and which `State` to show.

In [2]:
h1 = hb.Hubbard()

In [3]:
close_widgets(dir())
w1 = widgets.interactive(h1.show_basis, index=h1.basis_index, n=h1.n, s_up=h1.s_up, s_down=h1.s_down);

display(w1)

interactive(children=(BoundedIntText(value=6, description='State =', layout=Layout(width='3cm'), max=399, styl…

# Hopping Hamiltonian

In the Widget below one can play around with the number of sites $n$, spin up and spin down states, as well as the interaction strength $U$ and the hopping amplitude $t$ to explore the hamiltonian $H$. Note that only for $n \leq 4$ the complete matrix can be fully displayed. Note the different signs for the hopping terms.

In [12]:
#close previous widget to prevent slowing down calculations
close_widgets(dir())

w2 = widgets.interactive(h1.Show_H, u=h1.u, t=h1.t, n=h1.n, s_up=h1.s_up, s_down=h1.s_down);
display(w2)

interactive(children=(FloatSlider(value=2.6, continuous_update=False, description='$U=$', max=5.0, min=-5.0, r…

# Eigen-Energies

Our first goal is to diagonalize the Hamiltonian $H$ and calculate its Eigen-spectrum.

In the three widgets below the eigenvalues of $H$ are displayed as a function of either $U$ or $t$, while keeping the other one constant. One can again play around with the global parameters of the model ($n, s_\mathrm{up}, s_\mathrm{down}$) as well as change the hopping amplitude $t$ and the interaction strength $U$.

The last widget shows a combination of changes in $U$ and $t$. The non-interacting limit $U \to 0$ (weak coupling expansion) is shown on the left and the atomic limit $t \to 0 $ (strong coupling expansion) is shown on the right. The two plots are depicted in the same figure to emphasize the transition. Note the inverted $t$-axis on the right plot

The non-interacting limit is especially interesting, because we can find all eigenstates by simply diagonalizing the one particle Hamiltonian!

In [5]:
close_widgets(dir())
w3 = widgets.interactive(h1.Plot_Eigvals_Hu, u=h1.u_range, n=h1.n, s_up=h1.s_up, s_down=h1.s_down);

filename = set_filename("Eigenergies_U.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w3, name_widget=filename, output=h1.out, path="Figures/"))
 
d3 = HBox([Save_Figure_Button, filename, h1.out])
display(d3)
display(w3)

interactive(children=(FloatRangeSlider(value=(2.0, 7.0), continuous_update=False, description='$U=$', layout=L…

In [6]:
close_widgets(dir())
w4 = widgets.interactive(h1.Plot_Eigvals_Ht, t=h1.t_range, n=h1.n, s_up=h1.s_up, s_down=h1.s_down);

filename = set_filename("Eigenenergies_t.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w4, name_widget=filename, output=h1.out, path="Figures/"))

d4 = HBox([Save_Figure_Button, filename, h1.out])
display(d4)
display(w4)

interactive(children=(FloatRangeSlider(value=(0.0, 1.0), continuous_update=False, description='$t=$', max=5.0,…

In [6]:
close_widgets(dir())
w5 = widgets.interactive(h1.Plot_Eigvals_H, u=h1.t_range, t=h1.u_range, n=h1.n, s_up=h1.s_up, s_down=h1.s_down);

filename = set_filename("Eigenenergies_Ut.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w5, name_widget=filename, output=h1.out, path="Figures/"))

d5 = HBox([Save_Figure_Button, filename, h1.out])
display(d5)
display(w5)

interactive(children=(FloatRangeSlider(value=(0.0, 1.0), continuous_update=False, description='$t=$', max=5.0,…

# Expectation Values

After diagonalizing the Hamiltonian $H$ and obtaining the ground-state of the system $\ket{\psi_\mathrm{g}}$, we can calculate the ground-state expectation value of an operator $\hat O$ by

$$\large
	\expval{\hat{O}} = \mel{\psi_\mathrm{g}}{\hat O}{\psi_\mathrm{g}}
$$

## Average Double Occupation

Let us first consider the average double occupation of site $i$ $\expval{n_i^\mathrm{up} n_i^\mathrm{down}}$. The widget below shows the average double occupation in blue as a function of the interaction strength $U$. The dashed orange and green lines are the $U \to 0$ and $U \to \infty$ limit, respectively.
They are analytically given by

$$\large
	\expval{n_i^\mathrm{up} n_i^\mathrm{down}}_{U \to 0} = \frac{s_\mathrm{up}}{n} \frac{s_\mathrm{down}}{n} \quad \mathrm{and} \quad \expval{n_i^\mathrm{up} n_i^\mathrm{down}}_{U \to \infty} = \max\left(0, \frac{s_\mathrm{up} + s_\mathrm{down} - n}{n}\right)
$$

In [7]:
close_widgets(dir())
w6 = widgets.interactive(h1.Plot_ExpVal_nn, u=h1.u_range, n=h1.n, s_up=h1.s_up, s_down=h1.s_down);

filename = set_filename("Average_double_occupation.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w6, name_widget=filename, output=h1.out, path="Figures/"))

d6 = HBox([Save_Figure_Button, filename, h1.out])
display(d6)
display(w6)

interactive(children=(FloatRangeSlider(value=(0.0, 11.0), continuous_update=False, description='$U=$', layout=…

## Average Spin Moments

Next we consider some $z$-spin expectation values, $\expval{S_i}, \expval{S_i^2}$ and $\expval{\Delta S_i^2}$. The widget below shows these as a function of the interaction strength $U$.

In [8]:
close_widgets(dir())
w7 = widgets.interactive(h1.Plot_ExpVal_Sz, u=h1.u_range, n=h1.n, s_up=h1.s_up, s_down=h1.s_down);

filename = set_filename("Average_Spin.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w7, name_widget=filename, output=h1.out, path="Figures/"))

d7 = HBox([Save_Figure_Button, filename, h1.out])
display(d7)
display(w7)

interactive(children=(FloatRangeSlider(value=(0.0, 11.0), continuous_update=False, description='$U=$', layout=…

## Spin-Spin Correlations

We can also ask about non-local spin-spin correlations between sites $i$ and $j$. The widget below shows the spin-spin correlation as a function of the interaction strength $U$. Negative correlation implies spin on site $i$ and $j$ tends to align anti-parallel to each other, while positive correlation induces parallel spin alignment. Due to symmetry some combinations are equivalent, e.g. for $n=6$ we have $\expval{S_1S_2} = \expval{S_1S_6}$.

In [4]:
close_widgets(dir())
w8 = widgets.interactive(h1.Plot_ExpVal_SzSz, u=h1.u_range, n=h1.n, s_up=h1.s_up, s_down=h1.s_down);

filename = set_filename("Spin-Spin_Correlation.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w8, name_widget=filename, output=h1.out, path="Figures/"))

d8 = HBox([Save_Figure_Button, filename, h1.out])
display(d8)
display(w8)

interactive(children=(FloatRangeSlider(value=(2.0, 7.0), continuous_update=False, description='$U=$', layout=L…

## Perturbative Coupling

Finally, one can ask how the system responds to some small perturbation, e.g. an external magnetic field. The widget below shows the local susceptibility

$$\large
	\chi_\mathrm{loc} = 2 \sum_{m > g} \frac{\abs{\mel{\psi_m}{S_{iz}}{\psi_g}}^2}{E_m - E_g},
$$
as a function of the interaction strength $U$. Note that this formula only works for non-degenerate ground states!

In [8]:
close_widgets(dir())
w9 = widgets.interactive(h1.Plot_Local_Chi, u=h1.u_range, n=h1.n, s_up=h1.s_up, s_down=h1.s_down);

filename = set_filename("Local_Chi.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w9, name_widget=filename, output=h1.out, path="Figures/"))

d9 = HBox([Save_Figure_Button, filename, h1.out])
display(d9)
display(w9)

interactive(children=(FloatRangeSlider(value=(2.0, 7.0), continuous_update=False, description='$U=$', layout=L…

Like for the local susceptibility we can also ask how a staggered field

$$\large
S_\mathrm{staggered} = \sum_{i=1}^n (-1)^i S_{iz}
$$

affects the system. The widget below shows the systems response $\large \chi_\mathrm{staggered}$

In [9]:
close_widgets(dir())
w10 = widgets.interactive(h1.Plot_Chi_Staggered, u=h1.u_range, n=h1.n, s_up=h1.s_up, s_down=h1.s_down);

filename = set_filename("Staggered_Chi.pdf")
Save_Figure_Button.on_click(functools.partial(Click_Save_Figure, widget=w10, name_widget=filename, output=h1.out, path="Figures/"))

d10 = HBox([Save_Figure_Button, filename, h1.out])
display(d10)
display(w10)

interactive(children=(FloatRangeSlider(value=(2.0, 7.0), continuous_update=False, description='$U=$', layout=L…

In [ ]:
close_widgets(dir())
x = lambda u,  **kwargs: u
w99 = widgets.interactive(x,  {"manual": True}, u=h1.t_range, t=h1.u_range, n=h1.n, s_up=h1.s_up, s_down=h1.s_down)
w99